In [ ]:
import fiona
import math

In [ ]:
# fname = 'routes/Route from 2021-02-02 08 27_20210202082755.gpx'
# fname = 'routes/Route from 2021-02-03 08 20_20210203082009.gpx'
# fname = 'routes/Route from 2021-01-04 08 27_20210104082708.gpx'
fname = "routes/MyTracks_Route_from_2024-03-05_07_36_20240305073625.gpx"

## Fiona

Fiona reads and writes geographic data files

https://fiona.readthedocs.io/en/latest/README.html

In [ ]:
fiona.listlayers(fname)

We're interested in the tracks...

In [ ]:
layer = fiona.open(fname, layer='tracks')
lon_min, lat_min, lon_max, lat_max = layer.bounds

In [ ]:
geom = layer[0]  # first track? (my data only has one per file, AFAIK)
print(", ".join(geom.keys()))
print(geom['id'])
print(", ".join(geom['geometry'].keys()))

In [ ]:
coords = geom['geometry']['coordinates']

In [ ]:
from haversine import distance

d = 0.0

for p1, p2 in zip(coords[0], coords[0][1:]):
    d += distance(p1, p2)
    if d < 100:
        print(p1, p2, d)

print(f'total distance: {d:.1f}m ({d * 5.0 / 8000.0:.3f}m)')

In [ ]:
from shapely.geometry import shape

In [ ]:
data = {'type': 'MultiLineString', 'coordinates': coords}

shp = shape(data)

# Cartopy

https://scitools.org.uk/cartopy/docs/latest/index.html

Cartopy is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses.

crs: coordinate reference system

PlateCarree: Equirectangular projection [Wikipedia article](https://en.wikipedia.org/wiki/Equirectangular_projection)

Other projections are available, see https://scitools.org.uk/cartopy/docs/latest/crs/projections.html


In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


def make_map(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(9, 13),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
import cartopy.io.img_tiles as cimgt

request = cimgt.OSM()  # Open Street Map

pad_lat = 0.002
pad_lon = 0.002

extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

fig, ax = make_map(projection=request.crs)
ax.set_extent(extent)

img = ax.add_image(request, 14)
s = ax.add_geometries(shp, ccrs.PlateCarree(),
                      facecolor='none',
                      edgecolor='crimson',
                      linewidth=2)